In [1]:
import sys
print(sys.executable)
print(sys.version)

/opt/homebrew/opt/python@3.12/bin/python3.12
3.12.4 (main, Jun  6 2024, 18:26:44) [Clang 15.0.0 (clang-1500.3.9.4)]


In [6]:
import numpy as np
import pandas as pd

In [47]:
columns_desc = {'CUST_ID': 'Идентификация держателя кредитной карты (категориальный)',
 'BALANCE': 'Сумма остатка на счету для совершения покупок',
 'BALANCE_FREQUENCY': 'Как часто обновляется баланс, оценка от 0 до 1 (1 = часто обновляется, 0 = не обновляется)',
 'PURCHASES': 'Сумма покупок, сделанных со счета',
 'ONEOFF_PURCHASES': 'Максимальная сумма единовременной покупки',
 'INSTALLMENTS_PURCHASES': 'Сумма покупок, сделанных в рассрочку',
 'CASH_ADVANCE': 'Сумма аванса наличными, выданного пользователю',
 'PURCHASES_FREQUENCY': 'Как часто совершаются покупки, оценка от 0 до 1 (1 = часто покупается, 0 = не покупается)',
 'ONEOFF_PURCHASES_FREQUENCY': 'Как часто совершаются единовременные покупки (1 = часто покупается, 0 = не покупается)',
 'PURCHASES_INSTALLMENTS_FREQUENCY': 'Как часто совершаются покупки в рассрочку (1 = часто, 0 = не часто)',
 'CASH_ADVANCE_FREQUENCY': 'Как часто выплачивается аванс наличными',
 'CASH_ADVANCE_TRX': 'Количество транзакций, совершенных с использованием аванса наличными',
 'PURCHASES_TRX': 'Количество транзакций по покупкам',
 'CREDIT_LIMIT': 'Кредитный лимит пользователя по кредитной карте',
 'PAYMENTS': 'Сумма платежей, совершенных пользователем',
 'MINIMUM_PAYMENTS': 'Минимальная сумма платежей, совершенных пользователем',
 'PRC_FULL_PAYMENT': 'Процент полностью оплаченных платежей пользователем',
 'TENURE': 'Срок обслуживания по кредитной карте для пользователя'}

In [11]:
data = pd.read_csv('customer_credit_card_data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8950 entries, 0 to 8949
Data columns (total 18 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   CUST_ID                           8950 non-null   object 
 1   BALANCE                           8950 non-null   float64
 2   BALANCE_FREQUENCY                 8950 non-null   float64
 3   PURCHASES                         8950 non-null   float64
 4   ONEOFF_PURCHASES                  8950 non-null   float64
 5   INSTALLMENTS_PURCHASES            8950 non-null   float64
 6   CASH_ADVANCE                      8950 non-null   float64
 7   PURCHASES_FREQUENCY               8950 non-null   float64
 8   ONEOFF_PURCHASES_FREQUENCY        8950 non-null   float64
 9   PURCHASES_INSTALLMENTS_FREQUENCY  8950 non-null   float64
 10  CASH_ADVANCE_FREQUENCY            8950 non-null   float64
 11  CASH_ADVANCE_TRX                  8950 non-null   int64  
 12  PURCHA

In [12]:
data.isna().sum()

CUST_ID                               0
BALANCE                               0
BALANCE_FREQUENCY                     0
PURCHASES                             0
ONEOFF_PURCHASES                      0
INSTALLMENTS_PURCHASES                0
CASH_ADVANCE                          0
PURCHASES_FREQUENCY                   0
ONEOFF_PURCHASES_FREQUENCY            0
PURCHASES_INSTALLMENTS_FREQUENCY      0
CASH_ADVANCE_FREQUENCY                0
CASH_ADVANCE_TRX                      0
PURCHASES_TRX                         0
CREDIT_LIMIT                          1
PAYMENTS                              0
MINIMUM_PAYMENTS                    313
PRC_FULL_PAYMENT                      0
TENURE                                0
dtype: int64

In [48]:
for col in data.columns:
    print(f"{columns_desc.get(col, col):<100} {len(data[col].unique())}")

Идентификация держателя кредитной карты (категориальный)                                             8950
Сумма остатка на счету для совершения покупок                                                        8871
Как часто обновляется баланс, оценка от 0 до 1 (1 = часто обновляется, 0 = не обновляется)           43
Сумма покупок, сделанных со счета                                                                    6203
Максимальная сумма единовременной покупки                                                            4014
Сумма покупок, сделанных в рассрочку                                                                 4452
Сумма аванса наличными, выданного пользователю                                                       4323
Как часто совершаются покупки, оценка от 0 до 1 (1 = часто покупается, 0 = не покупается)            47
Как часто совершаются единовременные покупки (1 = часто покупается, 0 = не покупается)               47
Как часто совершаются покупки в рассрочку (1 = часто